In [1]:
import cv2 as cv
  
# Load the cascade  
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')  
  
# To capture video from webcam.   
cap = cv.VideoCapture(0)  

fourcc = cv.VideoWriter_fourcc(*'XVID')
video_out = cv.VideoWriter('test.mp4', fourcc, 20.0, (640,  480)) 
while True:  
    
    # Read the frame  
    _, img = cap.read()  
  
    # Convert to grayscale  
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  
  
    # Detect the faces  
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)  
  
    # Draw the rectangle around each face  
    for (x, y, w, h) in faces:  
        cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)  
    
    video_out.write(img)
    # Display  
    cv.imshow('Video', img)  
    
   
    # Stop if escape key is pressed  
    k = cv.waitKey(1) 
    # check if key is q then exit
    if k == ord("q"):
        break 
          
# Release the VideoCapture object  
cap.release()  
video_out.release()
cv.destroyAllWindows()
 

In [2]:
import cv2 as cv
import numpy as np


# Load the cascade  
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')  
  
# To capture video from existing video.   
cap = cv.VideoCapture('data/video.mov')  
  
while True:  
    # Read the frame  
    _, img = cap.read()  
  
    # Convert to grayscale  
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  
  
    # Detect the faces  
    faces = face_cascade.detectMultiScale(gray, 2,1 )  
  
    # Draw the rectangle around each face  
    for (x, y, w, h) in faces:  
        cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)  
  
    # Display  
    cv.imshow('Video', img)  
  
     
    k = cv.waitKey(30) 
    # check if key is q then exit
    if k == ord("q"):
        break
          
# Release the VideoCapture object  
cap.release()  
cv.destroyAllWindows()

In [8]:
import face_recognition
import cv2
import numpy as np
import os
import glob

faces_encodings = []
faces_names = []
cur_direc = os.getcwd()
path = os.path.join(cur_direc, 'data/faces/')
list_of_files = [f for f in glob.glob('data/faces/*.jpg')]
number_files = len(list_of_files)
names = list_of_files.copy()

for i in range(number_files):
    globals()['image_{}'.format(i)] = face_recognition.load_image_file(list_of_files[i])
    globals()['image_encoding_{}'.format(i)] = face_recognition.face_encodings(globals()['image_{}'.format(i)])[0]
    faces_encodings.append(globals()['image_encoding_{}'.format(i)])
# Create array of known names
    names[i] = names[i].replace(cur_direc,"")  
    faces_names.append(names[i])
    
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]
    if process_this_frame:
        face_locations = face_recognition.face_locations( rgb_small_frame)
        face_encodings = face_recognition.face_encodings( rgb_small_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces (faces_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance( faces_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = faces_names[best_match_index]
                face_names.append(name)
        #process_this_frame = not process_this_frame
        # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
    # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
    # Input text label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (0, 255, 0), 1)
        # Display the resulting image
    cv2.imshow('Video', frame)
    k = cv2.waitKey(1)
    # Hit 'q' on the keyboard to quit!
    if k == ord('q'):
        break
video_capture.release()  
cv2.destroyAllWindows()